# Inclus AI – Demo Summary Formulation

In [71]:
# import all neccesary libraries 

import pandas as pd
from openai import AzureOpenAI

In [ ]:
# innitialize client
client = AzureOpenAI(
    api_key="API KEY",
    api_version="2024-02-15-preview",
    azure_endpoint="ENDPOINT" 
)

In [ ]:
# read data
df = pd.read_excel("inclusData.xlsx")

In [ ]:
# Function to convert numbers to text
def convert_to_text(row):
    number=row['Answer']
    types=row['Dimension']
    if types=='Likelyhood':
        if number <=1:
            return 'Insignificant'
        elif 1<number<=2:
            return 'Small'
        elif 2<number<=3:
            return 'Medium'
        elif 3<=number<=4:
            return 'Likely'
        else:
            return 'Very likely'
    elif types=='Impact':
        if number <=1:
            return 'Insignificant'
        elif 1<number<=2:
            return 'Small'
        elif 2<number<=3:
            return 'Medium'
        elif 3<=number<=4:
            return 'Large'
        else:
            return 'Major'

# Apply the function to the 'Answer' column
df['Answer'] = df.apply(convert_to_text, axis=1)

Constructing prompt:

In [ ]:
formatted_strings = []

# Iterate over each row and format it as desired
for index, row in df.iterrows():
    formatted_string = f"ITEM: {row['Item']}, Comment: {row['Comment']}, Evaluation of {row['Dimension']} : {row['Answer']}"
    formatted_strings.append(formatted_string)

# Concatenate all formatted strings into one long string
long_string = '\n'.join(formatted_strings)

In [ ]:
# Here we define the 'instruction' for the 'system' part for the prompt. Here the data 'risk_data' is also injected into the prompt, but it could be presented elsewhere.
prompt_engineering_majick = 'The assistant is a risk analyst.'
company_context = 'Inclus is a Finnish scaleup company that provides a platform for doing collaborative risk analysis.'
data_context = 'The data that is provided below is gathered from multiple people within Inclus and concerns multiple risk events. Each risk event is described in a header above the individual assessments. Treat each comment equally.'

context_for_GPT = [
prompt_engineering_majick,
company_context,
data_context,
long_string,
]
system_content = '\n'.join(context_for_GPT) # goes into the prompt in 'messages' part in line '{"role": "system", "content": system_content}' later

Prompting ChatGpt to generate summaries

In [ ]:
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": "Write an executive summary of the results of the risk analysis. Expected output: Several paragraphs of high-quality text that correctly describe important features of the data, such as significant risks, high-quality comments, and so on."}
    ],
)